# COVID-19 Detection with Cascade R-CNN 🦠🩺

![Competition's Logo](https://storage.googleapis.com/kaggle-competitions/kaggle/26680/logos/header.png)

- *Author: Mariusz Wiśniewski*
- *Competition: [SIIM-FISABIO-RSNA COVID-19 Detection](https://www.kaggle.com/competitions/siim-covid19-detection)*

## Overview

This is a starter notebook that includes all of the tools required to preprocess the data, train the model, and generate a new submission.

### Libraries Used

- [PyTorch 🔥](https://pytorch.org)
- [MMDetection ☄️](https://mmdetection.readthedocs.io/en/latest/)
- [Weights&Biases 📈](https://wandb.ai/)

### References

- [Cascade R-CNN: High Quality Object Detection and Instance Segmentation 📃](https://arxiv.org/abs/1906.09756)
- [Deep Residual Learning for Image Recognition 📃](https://arxiv.org/abs/1512.03385)
- [SIIM MMDetection+CascadeRCNN+Weight&Bias 📓](https://www.kaggle.com/code/sreevishnudamodaran/siim-mmdetection-cascadercnn-weight-bias/notebook)

## Project Setup

### Installation of Additional Packages

In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection && pip install -e .

### Import Statements

In [ ]:
import sys

sys.path.insert(0, './mmdetection')

import os

from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmcv import Config

import numpy as np
from pathlib import Path
import random
import torch

In [ ]:
import warnings

warnings.filterwarnings('ignore')

### Weights & Biases Setup

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret('WANDB_API_KEY')

wandb.login(key=wandb_api)

### Imports and Seed Everything

In [ ]:
global_seed = 2022


def set_seed(seed=global_seed):
    """Sets the random seeds."""
    set_random_seed(seed, deterministic=False)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed()

## Prepare the MMDetection Config

### General Training Settings

In [ ]:
model_name = 'cascade_rcnn_x101_32x4d_fpn_1x'
fold = 0
job = 1

baseline_cfg_path = f'/kaggle/working/mmdetection/configs/cascade_rcnn/{model_name}_coco.py'
cfg = Config.fromfile(baseline_cfg_path)

# Folder to store model logs and weight files
job_folder = f'/kaggle/working/job{job}_{model_name}_fold{fold}'
cfg.work_dir = job_folder

# Set seed thus the results are more reproducible
cfg.seed = global_seed

if not os.path.exists(job_folder):
    os.makedirs(job_folder)

print('Job folder:', job_folder)

In [ ]:
# Set the number of classes
for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1

cfg.runner.max_epochs = 20  # Epochs for the runner that runs the workflow
cfg.total_epochs = 20

# Learning rate of optimizers. The LR is divided by 8 since the config file is originally for 8 GPUs
cfg.optimizer.lr = 0.02 / 8

## Learning rate scheduler config used to register LrUpdater hook
cfg.lr_config = dict(
    policy='CosineAnnealing',
    # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=False,
    warmup='linear',  # The warmup policy, also support `exp` and `constant`.
    warmup_iters=500,  # The number of iterations for warmup
    warmup_ratio=0.001,  # The ratio of the starting learning rate used for warmup
    min_lr=1e-6)

# config to register logger hook
cfg.log_config.interval = 20  # Interval to print the log

# Config to set the checkpoint hook, Refer to https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/checkpoint.py for implementation.
cfg.checkpoint_config.interval = 1  # The save interval is 1

cfg.gpu_ids = [0]
cfg.device = 'cuda'

### Configure Datasets for Training and Evaluation

In [ ]:
cfg.dataset_type = 'CocoDataset'  # Dataset type, this will be used to define the dataset
cfg.classes = ('Covid_Abnormality',)

cfg.data.train.img_prefix = '/kaggle/input/siim-covid19-512-images-and-metadata/train'  # Prefix of image path
cfg.data.train.classes = cfg.classes
cfg.data.train.ann_file = f'/kaggle/input/siim-covid19-coco-512x512-groupkfold/train_annotations_fold{fold}.json'
cfg.data.train.type = 'CocoDataset'

cfg.data.val.img_prefix = '/kaggle/input/siim-covid19-512-images-and-metadata/train'  # Prefix of image path
cfg.data.val.classes = cfg.classes
cfg.data.val.ann_file = f'/kaggle/input/siim-covid19-coco-512x512-groupkfold/val_annotations_fold{fold}.json'
cfg.data.val.type = 'CocoDataset'

cfg.data.test.img_prefix = '/kaggle/input/siim-covid19-512-images-and-metadata/train'  # Prefix of image path
cfg.data.test.classes = cfg.classes
cfg.data.test.ann_file = f'/kaggle/input/siim-covid19-coco-512x512-groupkfold/val_annotations_fold{fold}.json'
cfg.data.test.type = 'CocoDataset'

cfg.data.samples_per_gpu = 4  # Batch size of a single GPU used in testing
cfg.data.workers_per_gpu = 2  # Worker to pre-fetch data for each single GPU

### Setting Metric for Evaluation

In [ ]:
# The config to build the evaluation hook, refer to https://github.com/open-mmlab/mmdetection/blob/master/mmdet/core/evaluation/eval_hooks.py#L7 for more details.
cfg.evaluation.metric = 'bbox'  # Metrics used during evaluation

# Set the epoch intervel to perform evaluation
cfg.evaluation.interval = 1

cfg.evaluation.save_best='bbox_mAP_50'

### Prepare the Pre-processing & Augmentation Pipelines

In [ ]:
albu_train_transforms = [
    dict(type='ShiftScaleRotate', shift_limit=0.0625,
         scale_limit=0.15, rotate_limit=15, p=0.4),
    dict(type='RandomBrightnessContrast', brightness_limit=0.2,
         contrast_limit=0.2, p=0.5),
    dict(type='IAAAffine', shear=(-10.0, 10.0), p=0.4),
    #     dict(type='MixUp', p=0.2, lambd=0.5),
    dict(type='Blur', p=1.0, blur_limit=7),
    dict(type='CLAHE', p=0.5),
    dict(type='Equalize', mode='cv', p=0.4),
    dict(
        type='OneOf',
        transforms=[
            dict(type='GaussianBlur', p=1.0, blur_limit=7),
            dict(type='MedianBlur', p=1.0, blur_limit=7),
        ],
        p=0.4,
    )
]

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'])
]
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

### Weights & Biases Integration for Experiment Tracking and Logging

In [ ]:
cfg.log_config.hooks = [dict(type='TextLoggerHook'),
                        dict(type='WandbLoggerHook',
                             init_kwargs=dict(project=user_secrets.get_secret(
                                                  'WANDB_PROJECT'),
                                              entity=user_secrets.get_secret(
                                                  'WANDB_ENTITY'),
                                              name=f'8-mw-{model_name}-fold{fold}-job{job}',
                                              ))
                        ]

### Save Config File

In [ ]:
cfg_path = f'{job_folder}/job{job}_{Path(baseline_cfg_path).name}'
print(cfg_path)

# Save config file for later inference
cfg.dump(cfg_path)
print(f'Config:\n{cfg.pretty_text}')

### 🚀 Build Dataset and Start Training

In [ ]:
model = build_detector(cfg.model,
                       train_cfg=cfg.get('train_cfg'),
                       test_cfg=cfg.get('test_cfg'))
model.init_weights()

In [ ]:
datasets = [build_dataset(cfg.data.train)]

In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

In [ ]:
# Get the best epoch number
import json

from collections import defaultdict

log_file = f'{job_folder}/None.log.json'


# Source: mmdetection/tools/analysis_tools/analyze_logs.py
def load_json_logs(json_logs):
    # load and convert json_logs to log_dict, key is epoch, value is a sub dict
    # keys of sub dict is different metrics, e.g. memory, bbox_mAP
    # value of sub dict is a list of corresponding values of all iterations
    log_dicts = [dict() for _ in json_logs]
    for json_log, log_dict in zip(json_logs, log_dicts):
        with open(json_log, 'r') as log_file:
            for line in log_file:
                log = json.loads(line.strip())
                # skip lines without `epoch` field
                if 'epoch' not in log:
                    continue
                epoch = log.pop('epoch')
                if epoch not in log_dict:
                    log_dict[epoch] = defaultdict(list)
                for k, v in log.items():
                    log_dict[epoch][k].append(v)
    return log_dicts


log_dict = load_json_logs([log_file])
# [(print(inner['bbox_mAP']) for inner in item) for item in log_dict]
# [print(item) for item in log_dict[0]]
best_epoch = np.argmax([item['bbox_mAP'][0] for item in log_dict[0].values()]) + 1
best_epoch

In [ ]:
model_files = [f'{job_folder}/epoch_{best_epoch}.pth',
               cfg_path
               ]

# Create a new wnb run for saving models as artifacts
run = wandb.init(project=user_secrets.get_secret('WANDB_PROJECT'),
                 name=f'models_files_{model_name}_fold{fold}_job{job}',
                 entity=user_secrets.get_secret('WANDB_ENTITY'),
                 group='Artifact',
                 job_type='model-files')

artifact = wandb.Artifact(f'models_files_{model_name}_fold{fold}_job{job}', type='model')

for model_file in model_files:
    artifact.add_file(model_file)

run.log_artifact(artifact)
run.finish()

## Inference and Visualize Output

In [ ]:
from mmdet.apis import init_detector, inference_detector

import matplotlib.pyplot as plt
import pandas as pd
import cv2

In [ ]:
with open('../input/siim-covid19-coco-512x512-groupkfold/val_annotations_fold0.json') as f:
    val_ann = json.load(f)
image_paths = [item['file_name'] for item in val_ann['images'][:9]]

df_annotations = pd.read_csv('../input/siim-covid19-512-images-and-metadata/df_train_processed_meta.csv')

In [ ]:
def draw_bbox(image,
              box,
              label,
              color,
              label_size=0.5,
              alpha_box=0.3,
              alpha_label=0.6):
    overlay_bbox = image.copy()
    overlay_label = image.copy()
    output = image.copy()

    text_width, text_height = cv2.getTextSize(label.upper(),
                                              cv2.FONT_HERSHEY_SIMPLEX, label_size, 1)[0]
    cv2.rectangle(overlay_bbox, (box[0], box[1]), (box[2], box[3]),
                  color, -1)
    cv2.addWeighted(overlay_bbox, alpha_box, output, 1 - alpha_box, 0, output)

    cv2.rectangle(overlay_label, (box[0], box[1] - 7 - text_height),
                  (box[0] + text_width + 2, box[1]), (0, 0, 0), -1)
    cv2.addWeighted(overlay_label, alpha_label, output, 1 - alpha_label, 0, output)
    output = cv2.rectangle(output, (box[0], box[1]), (box[2], box[3]),
                           color, 2)
    cv2.putText(output, label.upper(), (box[0], box[1] - 5),
                cv2.FONT_HERSHEY_SIMPLEX, label_size, (255, 255, 255), 1, cv2.LINE_AA)
    return output

In [ ]:
checkpoint = f'{job_folder}/epoch_{best_epoch}.pth'

print('Loading weights from:', checkpoint)
cfg = Config.fromfile(cfg_path)
model = init_detector(cfg, checkpoint, device='cuda:0')

In [ ]:
new_size = (512, 512)
imgs_path = '/kaggle/input/siim-covid19-512-images-and-metadata/train'
threshold = 0.45

fig, axes = plt.subplots(3, 3, figsize=(19, 21))
fig.subplots_adjust(hspace=0.2, wspace=0.2)
axes = axes.ravel()

results_list = []

for idx, img_id in enumerate(image_paths):
    img_path = os.path.join(imgs_path, img_id)
    img = cv2.imread(img_path)
    result = inference_detector(model, img_path)
    results_filtered = result[0][result[0][:, 4] > threshold]
    bboxes = results_filtered[:, :4]
    scores = results_filtered[:, 4]
    results_list.append(result[0])

    for box in bboxes:
        img = draw_bbox(img, list(np.int_(box)), 'Covid_Abnormality',
                        (255, 243, 0))

    axes[idx].imshow(img, cmap='gray')
    axes[idx].set_title(img_id, size=18, pad=30)
    axes[idx].set_xticklabels([])
    axes[idx].set_yticklabels([])

### Interactively Visualize & Analyze Output in Dashboard

In [ ]:
run = wandb.init(project=user_secrets.get_secret('WANDB_PROJECT'),
                 name=f'images-{model_name}-fold{fold}-job{job}',
                 job_type='images')

class_id_to_label = {
    1: 'pred_covid_abnormality',
    2: 'GT_covid_abnormality'
}

wnb_images = []

for img_id, result in zip(image_paths, results_list):

    bboxes = result[:, :4]
    scores = result[:, 4]
    ann_dict = {'predictions': {
        'box_data': [],
        'class_labels': class_id_to_label
    },
        'ground_truth': {
            'box_data': [],
            'class_labels': class_id_to_label
        }
    }

    for box, score in zip(bboxes, scores):
        single_data = {
            # one box expressed in the default relative/fractional domain
            'position': {
                'minX': round(float(box[0]) / 512, 3),
                'maxX': round(float(box[2]) / 512, 3),
                'minY': round(float(box[1]) / 512, 3),
                'maxY': round(float(box[3]) / 512, 3),
            },
            'class_id': 1,
            'box_caption': class_id_to_label[1],
            'scores': {
                'confidence': float(score),
            }
        }
        ann_dict['predictions']['box_data'].append(single_data)

    image_annotations = df_annotations[df_annotations.id == img_id.strip('.png')]

    for idxx, row in image_annotations[['xmin', 'ymin', 'xmax', 'ymax']].iterrows():
        single_data = {
            # one box expressed in the default relative/fractional domain
            'position': {
                'minX': round(float(row[0]) / 512, 3),
                'maxX': round(float(row[2]) / 512, 3),
                'minY': round(float(row[1]) / 512, 3),
                'maxY': round(float(row[3]) / 512, 3),
            },
            'class_id': 2,
            'box_caption': class_id_to_label[2],
            'scores': {
                'confidence': 1.0,
            }
        }
        ann_dict['ground_truth']['box_data'].append(single_data)

    image = cv2.imread(os.path.join(imgs_path, img_id))
    wnb_images.append(wandb.Image(image, boxes=ann_dict))

wandb.log({f'images-{model_name}-fold{fold}-job{job}': wnb_images})

run.finish()
run

In [ ]:
!rm -rf mmdetection/